In [22]:
# google drive 연결
# url 파일이 드라이브 안에 있거나 코랩에 올려져 있어야 함
# 최초만 1회만 실행해도 됨
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# selenium & chrome driver 설치
# 최초 1회만 실행하기!
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://devel

In [4]:
# Selenium 설치 확인
!python --version
import selenium
print(selenium.__version__)

Python 3.10.12
4.25.0


In [285]:
#Selenium import
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException  # 예외 처리 추가
from selenium.webdriver.chrome.options import Options # user-agent 변경
#pandas import
import pandas as pd
#standard library
import time, os, random, sys, json
# chrome options
import chromedriver_autoinstaller
#BeautifulSoup import
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import re

# Auto Dict
Json 처리할 때 쓰려고 만든 dictionary 자료형 처리 클래스

In [298]:
class AutoDict:
    def __init__(self, existing_data=None):
        self.data = {}
        self.counter = 0  # 기본적으로 1부터 시작하지만 기존 데이터가 있으면 수정
        if existing_data:
            self.load_existing_data(existing_data)

    def load_existing_data(self, existing_data):
        # 기존 JSON 데이터를 로드하고 key에 맞춰 데이터를 채움
        self.data = {int(k): v for k, v in existing_data.items()}
        self.counter = max(self.data.keys()) + 1  # 다음 번호는 가장 큰 key + 1

    def add(self, value):
        # 가장 작은 빈 key를 찾아서 할당
        key = self._get_next_key()
        self.data[key] = value

    def remove(self, key):
        # key가 있는지 확인 후 삭제
        if key in self.data:
            del self.data[key]
            # key를 채우기 위해 재정렬
            self._fill_empty_keys()

    def update(self, key, value):
        # 특정 key에 데이터 업데이트
        self.data[key].update(value)

    def _get_next_key(self):
        # 빈 key를 찾음
        for i in range(1, len(self.data) + 2):
            if i not in self.data:
                return i

    def _fill_empty_keys(self):
        # key를 순차적으로 채우기 위해 데이터를 재정렬
        values = list(self.data.values())
        self.data = {i + 1: values[i] for i in range(len(values))}

    def __str__(self):
        return str(self.data)

# 데이터 불러오기
여기서 불러올 url 데이터랑 저장할 파일 경로 입력해주면 됨

In [258]:
data = pd.read_csv('/content/drive/Othercomputers/내 노트북/scrapping_darkpattern/data/test.CSV') # Input 파일(URL) 경로
output_file = "/content/drive/Othercomputers/내 노트북/scrapping_darkpattern/result/test.json" # Output 파일 경로

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Othercomputers/내 노트북/scrapping_darkpattern/data/test.CSV'

In [300]:
# 기존 데이터가 있는지 확인하고 로드
if os.path.isfile(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        try:
            output_data = json.load(f)  # 기존 데이터 읽기
            output_dataset = AutoDict(output_data)  # AutoDict 인스턴스 생성
        except json.JSONDecodeError:  # 파일이 비어 있을 경우 처리
            output_dataset = AutoDict()
else:
    output_dataset = AutoDict()

# function

In [301]:
def search_keyword(keywords, driver, url):
    # keyword가 포함된 block 불러와서 JSON 파일에 저장하는 함수

    # XPath OR 조건 생성
    xpath_conditions = " or ".join([f"contains(text(), '{keyword}')" for keyword in keywords])
    # 최종 XPath
    xpath = f"//*[{xpath_conditions}]"
    # Selenium을 활용해 요소 찾기
    elements = driver.find_elements(By.XPATH, xpath)

    # 데이터 생성
    #3. 데이터 가져오기
    find_div(elements, keywords, url)


In [302]:
# div 레이아웃 가져오기
def find_div(elements, keywords, url):
    i = 0
    for element in elements:
        try:
            print(f"지금 {i}번째 진행중")
            i += 1
            # 현재 요소를 포함하는 가장 가까운 div 찾기
            closest_div = element.find_element(By.XPATH, './ancestor::div[1]')
            html_code = closest_div.get_attribute('outerHTML')

            # CSS 스타일 가져오기
            css_styles = driver.execute_script("""
               const parent = arguments[0];
               const allElements = [parent, ...parent.querySelectorAll('*')]; // 모든 자식 요소를 선택
               const styles = {};
               allElements.forEach(element => {
                   const computedStyle = window.getComputedStyle(element);
                   const className = element.className.trim(); // 클래스 이름 가져오기
                   if (className) {
                       const styleObject = {};
                       for (let i = 0; i < computedStyle.length; i++) {
                           const property = computedStyle.item(i);
                           styleObject[property] = computedStyle.getPropertyValue(property);
                      }
                   styles[`.${className.replace(/\s+/g, '.')}`] = styleObject;
                   }
                });
                return styles;
            """, closest_div)

            # 스타일 출력
            style_output = ""
            for class_selector, styles in css_styles.items():  # 딕셔너리의 각 항목
                style_output += f"{class_selector} {{ \n"
                for property, value in styles.items():
                    style_output += f"  {property}: {value};\n"
                style_output += "}\n"

            # 일치하는 키워드 찾기
            matched_keywords = [keyword for keyword in keywords if keyword in html_code]

            # 결과 저장
            output_dataset.add({'HTML': html_code, 'CSS': style_output, 'Matched Keywords': matched_keywords, 'URL': url})

            # 결과 출력
            print("HTML Code:")
            print(html_code)

        except NoSuchElementException:
            print("가장 가까운 div 요소를 찾을 수 없습니다.")
        except Exception as e:
          print(f"오류 발생: {e}")

    return

In [303]:
#가장 작은 div 레이아웃 가져오기 (static _ beautifulSoup)
def find_static_div(elements):
    static_html = []

    if(elements == []):
      print("static : 찾을 수 없습니다.")

    for element in elements:
        try:
            closet_div = element.find_parent('div')
            if(closet_div):
                html_code = str(closet_div) #DOM 형식을 문자열로 변환
                print(f"HTML Code: {html_code}") #결과 출력
            else:
                closet_div = element.find_parent('script')
                if(closet_div):
                  html_code = str(closet_div) #DOM 형식을 문자열로 변환
                else:
                  print("static : 찾을 수 없습니다.")

            print(f"HTML Code: {html_code}") #결과 출력
            static_html.append(html_code) #결과 저장

        except NoSuchElementException:
            print("가장 가까운 div 요소를 찾을 수 없습니다.")

    return static_html

In [304]:
#거짓판별 함수 -> 거짓이 아니거나 오류(url 접속 오류, 요소가 없는 경우 등)일 경우 빈 문자열 저장
def search_false():
  print(f"길이 : {len(output_dataset.data)}, 더하기 : {len(output_dataset.data)+1}")
  for i in range(1,len(output_dataset.data)+1):
    url = output_dataset.data[i]['URL']
    dynamic_html = output_dataset.data[i]['HTML']
    matched_keywords = output_dataset.data[i]['Matched Keywords']
    print(f"URL : {url}, matched : {matched_keywords}")
    try :
      response = requests.get(url)
      response.raise_for_status()  # 오류 발생 시 예외 발생

      # BeautifulSoup으로 HTML 파싱
      static_html = BeautifulSoup(response.text, 'html.parser')
      time.sleep(random.randrange(10,20)) #로딩되는데 걸리는 시간(10초 이상)

      #dynamic_html에서 문장을 파싱(추출)
      soup = BeautifulSoup(dynamic_html, 'html.parser')
      dynamic_text = soup.get_text().strip()  # 모든 텍스트를 추출하고 공백 제거
      print(f"추출된 텍스트는 다음과 같습니다. --> {dynamic_text}")
      sentences = re.split(r'[.!?]+', dynamic_text) # 텍스트에서 문장 구분

      # matched_keywords와 일치하는 단어가 포함된 문장만 추출 (하나의 문장)
      final_sentence = next((sentence for sentence in sentences if any(keyword in sentence for keyword in matched_keywords)), None)

      # 최종 결과 출력
      if final_sentence:
          print(f"최종 문장은 다음과 같습니다. --> {final_sentence}")
      else:
          print("일치하는 문장이 없습니다.")

      #추출한 텍스트가 공백인 경우 None으로 저장
      if (final_sentence is None) or (final_sentence.strip() == ''):
          print("공백은 제외합니다!")
          output_dataset.update(i, {'Static_HTML': []})
          continue

      #정적 페이지의 텍스트 요소 전체와 동적 페이지 텍스트 비교해 일치하는 것 찾기
      static_elements = static_html.find_all(string=lambda text: final_sentence in text if text else False)
      print(f"static_elements : {static_elements}")

      static = find_static_div(static_elements)
      print(f"static : {static}")

      output_dataset.update(i, {'Static_HTML': static})

    except requests.exceptions.HTTPError as err: #url 연결 오류
      print(f"HTTP Error: {err}")
      output_dataset.update(i, {'Static_HTML':[]})
    except Exception as e:
      print(f"오류 발생: {e}")
      output_dataset.update(i, {'Static_HTML':[]})

  return

# Scrapper

In [305]:
# Chrome 옵션 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument("Mozilla/5.0")  # 원하는 User-Agent 문자열로 변경

In [306]:
driver = webdriver.Chrome(options=chrome_options) # chrome driver의 경로가 파일 경로와 같아야 함

In [307]:
# Keyword 설정
price_disrupt = ["실적", "조건", "신한카드", "삼성카드", "KB국민카드", "현대카드", "롯데카드", "우리카드", "하나카드", "BC카드", "NH농협카드"]
hidden_info = [ "세금", "수수료", "청소료", "부가세", "서비스세", "관세", "팁", "관리비", "청소비", "청소료", "정리비", "운영비", "중개료", "중개비"]
emotional_lang = ["마지막", "포기", "정말로", "놓치지", "기회", "지금", "한정", "수량", "곧", "마감", "임박", "가장", "너무", "망설"]
disguised_ad = ["AD", "ad", "Ad"]
trap_question = ["않으"]
false_sale = ["1+1 할인", "1+1할인", "원쁠원 할인", "원플원 할인", "원 쁠 원 할인", "원 플 원 할인"]
user_alert = ["보고있", "보는중", " 보고 있", "보는 중"]
stock_alert = ["남음", "남았", "개 남음"]

keywords_dict = [price_disrupt, hidden_info, emotional_lang, disguised_ad, trap_question, false_sale, user_alert, stock_alert]

In [308]:
for url in data['URL']:

    #1.url open
    print(url)
    driver.get(url)

    # time.sleep(random.randrange(10, 20)) # 10초 이상 기다려서 로딩되는데 걸리는 시간을 주기
    time.sleep(5)
    for keyword in keywords_dict:
        search_keyword(keyword, driver, url) # element 불러와서 output_dataset에 저장


https://www.kurly.com/main
지금 0번째 진행중
오류 발생: Message: javascript error: element.className.trim is not a function
  (Session info: chrome=130.0.6723.91)
Stacktrace:
#0 0x5b9d8e26c13a <unknown>
#1 0x5b9d8dd87460 <unknown>
#2 0x5b9d8dd8d193 <unknown>
#3 0x5b9d8dd8f0e4 <unknown>
#4 0x5b9d8de17a7c <unknown>
#5 0x5b9d8ddf8012 <unknown>
#6 0x5b9d8de16af6 <unknown>
#7 0x5b9d8ddf7db3 <unknown>
#8 0x5b9d8ddc6c10 <unknown>
#9 0x5b9d8ddc7bee <unknown>
#10 0x5b9d8e2383cb <unknown>
#11 0x5b9d8e23c368 <unknown>
#12 0x5b9d8e225eec <unknown>
#13 0x5b9d8e23cee7 <unknown>
#14 0x5b9d8e20b13f <unknown>
#15 0x5b9d8e25a858 <unknown>
#16 0x5b9d8e25aa20 <unknown>
#17 0x5b9d8e26afb6 <unknown>
#18 0x7c81c9c7aac3 <unknown>

지금 0번째 진행중
가장 가까운 div 요소를 찾을 수 없습니다.
지금 1번째 진행중
가장 가까운 div 요소를 찾을 수 없습니다.
지금 2번째 진행중
가장 가까운 div 요소를 찾을 수 없습니다.
https://www.kurly.com/collection-groups/market-sales-group?page=1&collection=market-sales-main1&per_page=96&sorted_type=1&filters=
지금 0번째 진행중
오류 발생: Message: javascript error: element

In [309]:
#거짓 판별 함수
search_false()

길이 : 2, 더하기 : 3
URL : https://www.kurly.com/collection-groups/market-sales-group?page=1&collection=market-sales-main1&per_page=96&sorted_type=1&filters=, matched : ['마감']
추출된 텍스트는 다음과 같습니다. --> 알뜰쇼핑알뜰쇼핑마감세일반값세일신선&정육만원의행복장보기초특가
최종 문장은 다음과 같습니다. --> 알뜰쇼핑알뜰쇼핑마감세일반값세일신선&정육만원의행복장보기초특가
static_elements : []
static : 찾을 수 없습니다.
static : None
URL : https://m.bunjang.co.kr/categories/930200?&order=date, matched : ['한정']
추출된 텍스트는 다음과 같습니다. --> $32.69한정판 레고 40601 매지스토의 마법 작업장
최종 문장은 다음과 같습니다. --> 69한정판 레고 40601 매지스토의 마법 작업장
static_elements : []
static : 찾을 수 없습니다.
static : None


In [310]:
# JSON으로 바꿔서 저장하기
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_dataset.data, f, ensure_ascii=False, indent=4)

In [311]:
driver.quit()